In [1]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
import pickle as pkl
import matplotlib.pyplot as plt
import random
import pyfixest as pf
import statsmodels.api as sm
import itertools as it
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
regression_data_full = pd.read_csv("../data/regression/gdp_regression_data.csv").dropna().reset_index()

In [3]:
regression_data_full

,index,Unnamed: 0,country,year,fd_ln_gdp,wildfire,drought,heat_wave,wildfire_drought,wildfire_heat_wave,...,YEM_incremental_effect_3,ZAF_incremental_effect_1,ZAF_incremental_effect_2,ZAF_incremental_effect_3,ZMB_incremental_effect_1,ZMB_incremental_effect_2,ZMB_incremental_effect_3,ZWE_incremental_effect_1,ZWE_incremental_effect_2,ZWE_incremental_effect_3
0,63,63,AFG,1961,0.001191,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,64,64,AFG,1962,-0.024206,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,65,65,AFG,1963,0.296924,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,66,66,AFG,1964,0.041662,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,67,67,AFG,1965,0.207628,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,12972,12972,ZWE,2018,0.643695,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,58,3364,195112
8469,12973,12973,ZWE,2019,-0.467445,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,59,3481,205379
8470,12974,12974,ZWE,2020,-0.035195,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,60,3600,216000
8471,12975,12975,ZWE,2021,0.256415,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,61,3721,226981


In [4]:
enc = OrdinalEncoder()
ordered_country_list = list(dict.fromkeys(regression_data_full.country))
enc.fit(np.array(ordered_country_list).reshape(-1,1))
regression_data_full["encoded_country"] = [int(val) for val in enc.transform(np.array(regression_data_full.country).reshape(-1,1))]
fixed_effects = ["encoded_country","year"]
columns_to_center = [
        "fd_ln_gdp",
        "temp_unweighted",
        "temp_unweighted_2",
        "precip_unweighted",
        "precip_unweighted_2"
    ]
centered_data = pf.estimation.demean(
    np.array(regression_data_full[columns_to_center]), 
    np.array(regression_data_full[fixed_effects]), 
    np.ones(len(regression_data_full))
)[0]
centered_data = pd.DataFrame(centered_data, columns=columns_to_center)
centered_data = pd.concat([regression_data_full.country, regression_data_full.year, centered_data], axis=1).reset_index()

/tmp/ipykernel_27624/600589821.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  regression_data_full["encoded_country"] = [int(val) for val in enc.transform(np.array(regression_data_full.country).reshape(-1,1))]
/home/hayden_freedman/pymc_dev.venv/lib/python3.10/site-packages/pyfixest/estimation/demean_.py:229: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  n_threads = nb.get_num_threads()


In [7]:
# group_means = dict.fromkeys(columns_to_center, dict.fromkeys(fixed_effects,{}))
# group_means = {
#     "fd_ln_gdp":{"country":{},"year":{},"overall":0},
#     "temp_unweighted":{"country":{},"year":{},"overall":0},
#     "temp_unweighted_2":{"country":{},"year":{},"overall":0},g
#     "precip_unweighted":{"country":{},"year":{},"overall":0},
#     "precip_unweighted_2":{"country":{},"year":{},"overall":0}
# }
# for var in group_means.keys():
#     for fe in group_means[var].keys():
#         if fe != "overall":
#             for group in set(regression_data_full[fe]):
#                 group_means[var][fe][group] = np.mean(regression_data_full.loc[(regression_data_full[fe] == group)][var])
#     group_means[var]["overall"] = np.mean(regression_data_full[var])

# centered_data = copy.deepcopy(regression_data_full)
# for var in group_means.keys():
#     demeaned_var = []
#     for row in centered_data.itertuples():
#         demeaned_var.append(getattr(row,var) - group_means[var]["country"][row.country] - group_means[var]["year"][row.year])# + group_means[var]["overall"])
#     centered_data[var+"_dm"] = demeaned_var

In [15]:
regression = pf.feols("fd_ln_gdp ~ temp_unweighted + temp_unweighted_2 + precip_unweighted + precip_unweighted_2 | country + year", data=regression_data_full)
print(regression.tidy())
print(regression.predict())

                         Estimate    Std. Error   t value  Pr(>|t|)  \
Coefficient                                                           
temp_unweighted      9.333517e-03  5.933456e-03  1.573032  0.117627   
temp_unweighted_2   -1.505879e-04  1.622831e-04 -0.927934  0.354797   
precip_unweighted    1.034436e-04  1.417241e-04  0.729894  0.466490   
precip_unweighted_2 -1.393155e-07  3.574224e-07 -0.389778  0.697203   

                             2.5%         97.5%  
Coefficient                                      
temp_unweighted     -2.381769e-03  2.104880e-02  
temp_unweighted_2   -4.710070e-04  1.698312e-04  
precip_unweighted   -1.763829e-04  3.832701e-04  
precip_unweighted_2 -8.450266e-07  5.663955e-07  
[ 0.02192069  0.01255798  0.04599324 ... -0.10157101  0.08617368
  0.03324222]


In [16]:
country_fe_dict = regression.fixef()["C(country)"]
year_fe_dict = regression.fixef()["C(year)"]
coef_vals = regression.tidy()["Estimate"]
secondrow = regression_data_full.iloc[1]
print(
    (secondrow.temp_unweighted * coef_vals["temp_unweighted"]) + 
    (secondrow.temp_unweighted_2 * coef_vals["temp_unweighted_2"]) + 
    (secondrow.precip_unweighted * coef_vals["precip_unweighted"]) +
    secondrow.precip_unweighted_2 * coef_vals["precip_unweighted_2"] +
    country_fe_dict[secondrow.country] + year_fe_dict[str(secondrow.year)]
)

0.01255797950653202


In [20]:
regression = pf.feols("fd_ln_gdp ~ temp_unweighted + temp_unweighted_2 + precip_unweighted + precip_unweighted_2 | 0", data=centered_data)
print(regression.tidy())
print(regression.predict(regression_data_full) + country_fe_dict[secondrow.country] + year_fe_dict[str(secondrow.year)])

                         Estimate    Std. Error       t value  Pr(>|t|)  \
Coefficient                                                               
Intercept           -1.487667e-18  1.639624e-03 -9.073218e-16  1.000000   
temp_unweighted      9.333517e-03  5.604373e-03  1.665399e+00  0.095870   
temp_unweighted_2   -1.505879e-04  1.804024e-04 -8.347334e-01  0.403891   
precip_unweighted    1.034436e-04  2.062642e-04  5.015102e-01  0.616025   
precip_unweighted_2 -1.393155e-07  5.798194e-07 -2.402740e-01  0.810124   

                         2.5%         97.5%  
Coefficient                                  
Intercept           -0.003214  3.214064e-03  
temp_unweighted     -0.001652  2.031946e-02  
temp_unweighted_2   -0.000504  2.030449e-04  
precip_unweighted   -0.000301  5.077718e-04  
precip_unweighted_2 -0.000001  9.972720e-07  
[0.01876958 0.01255798 0.02463134 ... 0.06465612 0.06170359 0.06319413]


In [13]:
coef_vals = regression.tidy()["Estimate"]
secondrow = centered_data.iloc[1]
print(
    (secondrow.temp_unweighted * coef_vals["temp_unweighted"]) + 
    (secondrow.temp_unweighted_2 * coef_vals["temp_unweighted_2"]) + 
    (secondrow.precip_unweighted * coef_vals["precip_unweighted"]) + 
    (secondrow.precip_unweighted_2 * coef_vals["precip_unweighted_2"]) +
    coef_vals["Intercept"] #+ 
    # country_fe_dict[secondrow.country] + year_fe_dict[str(secondrow.year)]
    )

-0.006688253197881225


In [23]:
coef_vals = regression.tidy()["Estimate"]
secondrow = centered_data.iloc[1]
print(
        (secondrow.temp_unweighted * coef_vals["temp_unweighted"] - group_means["temp_unweighted"]["country"][secondrow.country] * coef_vals["temp_unweighted"] - group_means["temp_unweighted"]["year"][secondrow.year] * coef_vals["temp_unweighted"]) + 
        (secondrow.temp_unweighted_2 * coef_vals["temp_unweighted_2"] - group_means["temp_unweighted_2"]["country"][secondrow.country] * coef_vals["temp_unweighted_2"] - group_means["temp_unweighted_2"]["year"][secondrow.year] * coef_vals["temp_unweighted_2"]) + 
        (secondrow.precip_unweighted * coef_vals["precip_unweighted"] - group_means["precip_unweighted"]["country"][secondrow.country] * coef_vals["precip_unweighted"] - group_means["precip_unweighted"]["year"][secondrow.year] * coef_vals["precip_unweighted"]) + 
        (secondrow.precip_unweighted_2 * coef_vals["precip_unweighted_2"]- group_means["precip_unweighted_2"]["country"][secondrow.country] * coef_vals["precip_unweighted_2"] - group_means["precip_unweighted_2"]["year"][secondrow.year] * coef_vals["precip_unweighted_2"]) + 
        coef_vals["Intercept"] +
        group_means["fd_ln_gdp"]["country"][secondrow.country] +
        group_means["fd_ln_gdp"]["year"][secondrow.year]
    )

-0.13180734272224473


In [144]:
centered_data.iloc[1],regression_data_full.iloc[1]["temp_unweighted"]

(index                           1
 fd_ln_gdp               -0.024206
 country                       AFG
 year                         1962
 temp_unweighted         -0.946094
 temp_unweighted_2      -16.260941
 precip_unweighted       -2.706238
 precip_unweighted_2    191.121679
 Name: 1, dtype: object,
 10.351284281412916)

In [145]:
import copy
def _subtract_weighted_group_mean(data, group_ids, var):
    _group_weighted_sums = {}
    x = copy.deepcopy(data)
    for i in range(x.size):
        id = group_ids[i]
        if id not in _group_weighted_sums:
            _group_weighted_sums[id] = 0
        _group_weighted_sums[id] += x[i]

    for i in range(x.size):
        id = group_ids[i]
        x[i] -= _group_weighted_sums[id] / len(list(x.loc[regression_data_full[var] == id]))

    return x

x = _subtract_weighted_group_mean(regression_data_full.temp_unweighted, regression_data_full.year, "year")
print(_subtract_weighted_group_mean(x, regression_data_full.encoded_country, "encoded_country"))

0      -0.327078
1      -1.187397
2       0.774312
3      -0.441213
4       1.231995
          ...   
8468    0.907684
8469    0.926611
8470    0.701581
8471    0.581026
8472    0.537584
Name: temp_unweighted, Length: 8473, dtype: float64
